## Introduction 

In this notebook we will go through a CRF only named-entity recognition implementation based on finance corpus. The following would be the sequence of the notebook:
<br>
1. Loading the dataset into a dataframe
2. Data Preprocessing
3. Extract features from the sentences (Feature Engineering)
4. Training a Condtional Random Field model
5. Evaluating the trained CRF model
6. Optimising the hyperparameters 

## Import the required libraries

In [ ]:
import pandas as pd
import numpy as np 

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import learning_curve
from sklearn.model_selection import train_test_split

from sklearn_crfsuite import CRF
from sklearn.metrics import make_scorer
from sklearn_crfsuite import metrics
from sklearn.exceptions import UndefinedMetricWarning 

import warnings
import nltk
import math
import sys

## Data Preprocessing


First, the data is loaded into a Pandas DataFrame. This can be done easily using the read_csv function, specifying that the separator is a space. It's also useful to keep the blank lines, which are helpful later for determining the sentence breaks. <br>
<br>
Once the data is loaded into a DataFrame, the easy access we have to columns allows a couple of useful things to be done - group the data by the "ne" column to see the distributions of each tag, and extract the classes (disregarding 'O' and blank lines with NaN values) as a list

### Parts of Speech Tag Generation

In [ ]:
# Read the NER data keeping blank lines and adding columns
ner_data = pd.read_csv("../Data/tag1.csv", skip_blank_lines=False, encoding="utf-8", index_col=None)
ner_data.columns = ["Token", "NE"]

POS_tags =  nltk.pos_tag(ner_data["Token"])
POS_List = []

for w in POS_tags:
    POS_List.append(w[1])
    
ner_data["POS"] = POS_List
    
print(ner_data)

### Visualize Tag Distribution

In [ ]:
tag_distribution = ner_data.groupby("NE").size().reset_index(name='counts')
print(tag_distribution)

Now filtering the classes of Named Entity that we do not require in this analysis

In [ ]:
classes = list(filter(lambda x: x not in ["O", np.nan], list(ner_data["NE"].unique())))
print(classes)

### Extract sentences from dataset


Next, sentences need to be extracted from the data - it's useful to have the sentences as a list of lists, with each sublist containing the token, POS tag, and NE label for every word token in the sentence

In [ ]:
# Create a sentences dictionary and an initial single sentence dictionary
sentences, sentence = [], []
# Create a progress bar
# pbar = pyprind.ProgBar(len(ner_data))
# For each row in the NER data...
for index, row in ner_data.iterrows():
    # If the row is empty (no string in the token column)
    if type(row["Token"]) != str:
        # If the current sentence is not empty, append it to the sentences and create a new sentence
        if len(sentence) > 0:
            sentences.append(sentence)
            sentence = []
    # Otherwise...
    else:
        # If the row does not indicate the start of a document, add the token to the current sentence
        if type(row["Token"]) != float and type(row["POS"]) != float and type(row["NE"]) != float:
            if not row["Token"].startswith("-DOCSTART-"):
                sentence.append([row["Token"], row["POS"], row["NE"]])
    #pbar.update()
print ("Done")

## Feature Engineering

Now we are going to define a function which would allow us to extract the word features in the sentence. This includes the following:
<br>
1. Current Parts of Speech Tags
2. Previous and Next Parts of Speech Tags
3. Current Words
4. Previous Words
5. Next Words

<br>For now, we have avoided chunking however a little internet research shows us that chunking indeed can improve the accuracy and sensitivity of the model

In [ ]:
def extractWordFeatures(sentence, iterator):
    POS_Tag = sentence[i][1]
    Token = sentence[i][0]

    # Aggregating a feuature dicitonary based on the features of the current POS and word
    
    featureDict = { "POS[:2]" : POS[:2],
                 "POS" : POS,
                 "Token.isdigit()" : Token.isdigit(),
                 "Token.istitle()" : Token.istitle(),
                 "Token.isupper()" : Token.isupper(),
                 "Token[-2:]" : Token[-2:],
                 "Token[-3:]" : Token[-3:],
                 "Token.lower()" : Token.lower(),
                 "bias" : 1.0,
    }
    
    if iterator > 1:
        previousWord = sentence[i-1][0]
        previousPosTag = sentence[i-1][1]
        
        # Add characteristics of the sentence's previous word and POS to the feature dictionary
        featureDict.update({ "-1:Token.lower()": previousWord.lower(),
                          "-1:Token.istitle()": previousWord.istitle(),
                          "-1:Token.isupper()": previousWord.isupper(),
                          "-1:POS": previousPostag,
                          "-1:POS[:2]": previousPostag[:2],
                        })
        
    # Add "Beginning of Sentence" at the start of the dictionary    
    else:
        featureDict["BOS"] = True
    
    if iterator < len(sentence)-1:
        nextWord = sentence[i+1][0]
        nextPos = sentence[i+1][1]
        # Add characteristics of the sentence's previous next and POS to the feature dictionary
        featureDict.update({ "+1:Token.lower()": nextWord.lower(),
                          "+1:Token.istitle()": nextWord.istitle(),
                          "+1:Token.isupper()": nextWord.isupper(),
                          "+1:POS": nextPos,
                          "+1:POS[:2]": nextPos[:2],
                        })
        
    else:
        features["EOS"] = True
return features    